### Import Statment


In [30]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [31]:
# Custom imports:
from story_outline_generation import StoryOutlineGenerator
from expert_interview_chain import InterviewChain
from characters_generation import CharacterGenerator
# from story_generation import StoryGenerator

In [32]:
# Hard core input
input_message = "Generate a adventure story for James, Joey, Emma, and Lily. The story should be about a treasure hunt in the jungle. The story should be set in the 19th century."
genre = "Romantic"

### Interview Question


In [33]:
# Ask Details Question about the stories
interview_chain = InterviewChain(topic=input_message, genre=genre)
interview_questions = interview_chain()

In [34]:
for question in interview_questions.questions:
    print(f"Answer the following question: {question.question}\n", flush=True)
    answer = input(f"Answer the following question: {question.question}\n")
    print('------------------------------------------')
    question.answer = answer

Answer the following question: What are the names of the main characters in the adventure story set in the 19th century jungle?

------------------------------------------
Answer the following question: Can you describe the relationships between James, Joey, Emma, and Lily in the story?

------------------------------------------
Answer the following question: What backgrounds or personalities do James, Joey, Emma, and Lily have that drive them to embark on a treasure hunt in the jungle?

------------------------------------------
Answer the following question: Where is the jungle located in the 19th-century setting of the story?

------------------------------------------
Answer the following question: Could you share a specific scene from the adventure story involving James, Joey, Emma, and Lily during their treasure hunt in the jungle?

------------------------------------------


### Stories Outline


In [35]:
blog_outline_generator = StoryOutlineGenerator(
    input=input_message, genre=genre, interview_questions_and_answers=interview_questions)
outline_result = blog_outline_generator.generate_outline()

Generating the Stories Outline...
---
Finished generating the outline!
---


In [36]:

for chapter in outline_result.chapters:
    print(f"Chapter {chapter.chapter_number}: {chapter.chapter_title}")
    print('------------------------------------------')

Chapter 1: The Call to Adventure
------------------------------------------
Chapter 2: Into the Heart of the Jungle
------------------------------------------
Chapter 3: Love and Loss
------------------------------------------


### Character Bilography Generation


In [37]:
from characters_generation import CharacterGenerator

In [38]:
character_generator = CharacterGenerator(
    input=input_message, genre=genre, interview_questions_and_answers=interview_questions)
character_result = character_generator.generate_character()

Generating the Characters...
---
Finished generating the characters!
---


In [39]:
for character in character_result.characters:
    print(character)

name='James' appearance="A tall, muscular black man with short curly hair and deep brown eyes, often seen wearing a rugged explorer's outfit suitable for the jungle." biography='James is a passionate adventurer driven by his love for Joey. He is determined and courageous, always ready to face the challenges of the jungle. His love for Joey motivates him to embark on the treasure hunt, hoping to find a fortune that could secure their future together.'
name='Joey' appearance='A petite white woman with long blonde hair and bright blue eyes, dressed in practical yet stylish attire for jungle exploration.' biography="Joey is a spirited and intelligent woman who shares a deep bond with James. Her adventurous spirit and curiosity about the world lead her to join the treasure hunt. She is resourceful and quick-witted, often coming up with solutions to the group's problems."
name='Emma' appearance='A young Asian girl with straight black hair and warm brown eyes, wearing a simple yet functional 

### Character Image Generation


In [40]:
from openai import OpenAI

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))


def generate_character_image(name, appearance, biography, genre="realistic"):
    # Create a detailed prompt for DALL-E
    # prompt = f"A character portrait in {genre} style. The character is {name}, who is {biography}. The image should be detailed and high quality, showing the character's distinctive features and personality."
    prompt = f"""A highly detailed character portrait in {genre} style.
    The character is {name}, with a background in {biography}. with this appearance: {appearance}.
    - The character image must be look like what describe in appearance"""

    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )

    image_url = response.data[0].url

    return image_url

In [41]:
character_image = []
for character in character_result.characters:
    image_url = generate_character_image(
        name=character.name, appearance=character.appearance, biography=character.biography)
    character_image.append(image_url)

In [ ]:
print(f"The image is ready! The filepath is {character_image[0]}")

In [42]:
from IPython.display import Image, display

for image_url in character_image:
    display(Image(url=image_url))

# Adjust images


In [28]:
character_image = []
for i, character in enumerate(character_result.characters):
    if i == 0:
        continue
    # Append additional text to the appearance
    character.appearance += " Remove the spectacles and add a hat."

for character in character_result.characters:
    # Generate the character image
    image_url = generate_character_image(
        name=character.name,
        appearance=character.appearance,
        biography=character.biography
    )
    character_image.append(image_url)

In [43]:
from IPython.display import Image, display

for image_url in character_image:
    display(Image(url=image_url))

### Stories Generation


In [44]:
from story_generation import StoryGenerator

In [45]:
story_gen = StoryGenerator(topic=input_message, outline=outline_result,
                           questions_and_answers=interview_questions, characters=character_result)

# Create the stories (This will take couple of minutes, because generate 3 chapters)
stories = story_gen.generate_stories()

Generating the story...
---
Finished generating the story!
---


In [46]:
for story in stories:
    print("====================================================================")
    print(story)

# Chapter 1: The Call to Adventure

The dense jungle of South Africa rustled with life as the 19th century sun beat down on its lush foliage. In this untamed wilderness, four individuals were about to embark on a thrilling treasure hunt that would test their courage, friendship, and determination.

### The Characters

#### James
A tall, muscular black man with short curly hair and deep brown eyes, James was an experienced adventurer. Clad in a rugged explorer's outfit suitable for the jungle, he exuded an air of confidence and determination. James was driven by his love for Joey, a petite white woman with long blonde hair and bright blue eyes.

#### Joey
Joey, a spirited and intelligent woman, shared a deep bond with James. Her adventurous spirit and curiosity about the world made her the perfect companion for this perilous journey. Dressed in practical yet stylish attire for jungle exploration, Joey's resourcefulness and quick-wittedness often proved invaluable to the group.

#### Emm

### Create a summary of the story


In [22]:
from summary_chain import StorySummarizer

In [23]:
summarizer = StorySummarizer(story=stories)

story_summary = summarizer.summarize_story()

Generating the Story Summary...
---
Finished generating the summary!
---


In [24]:
for story in story_summary:
    print(story)

('summary', "In a bustling coffee shop in China, Julie, a dedicated coffee worker, and Romeo, a sophisticated gentleman, experience a chance encounter that sparks an unexpected romance. Their paths cross when Julie accidentally hands Romeo the wrong coffee, leading to a moment of shared amusement and a deepening connection. Despite their different backgrounds, they find solace in each other's company, sharing stories and laughter. This serendipitous meeting marks the beginning of an extraordinary love story, as they navigate the complexities of their newfound relationship, discovering the power of love, forgiveness, and second chances.")


### Create title selection


In [25]:
from title_selection_chain import TitleGenerator

In [26]:

title_generator = TitleGenerator(summary=stories)

candidate_titles = title_generator.generate_titles()

Generating potential titles for the story...
---
Finished generating the title candidates!
---


In [27]:
for title in candidate_titles.titles:
    print(title)

Serendipity in a Coffee Cup
The Aroma of Fate
A Love Brewed by Chance
Mistaken Orders, Perfect Connection
The Coffee Shop Chronicles: A Love Story


### Trash Bin


In [ ]:
import getpass

api_key = getpass.getpass("Enter your Stability API key: ")
os.environ["STABILITY_API_KEY"] = api_key

In [ ]:
from image_generation import generate_character_image

In [ ]:
character_image = []
for character in character_result.characters:
    image_url = generate_character_image(
        name=character.name, appearance=character.appearance, biography=character.biography)
    character_image.append(image_url)

In [ ]:
from IPython.display import Image, display

for image_url in character_image:
    display(Image(url=image_url))